In [1]:
from codecs import encode
from pynq import Overlay, mmio, allocate
import time
from scipy.io import wavfile
import numpy as np
import os

In [65]:
overlay = Overlay('/home/xilinx/i2s_to_mp3/i2s_to_mp3.bit') # load the PL

In [2]:
# PL register map
base_addr = 0x43c00000
version_offset = 0
capture_start_offset = 4 # write 1 to bit 0 to reset and start a capture, self-clearing
capture_addr_offset = 8 # address of array in processor memory
capture_size_offset = 12
transfer_done_offset = 16
srst_i2s_offset = 20
i2s_config_offset = 24
lrck_cnt_offset = 28
bck_cnt_offset = 32

reg_space = mmio.MMIO(base_addr, 36)

In [3]:
reg_space.read(version_offset) # check version number

19

In [4]:
sr = 44100
capture_size = sr*60

In [5]:
# allocate 65536 32-bit words of shared mem for captured audio samples
audio_buffer = allocate(shape=(capture_size,), dtype='u4')
# sync the result buffer memory to PL
audio_buffer.flush()
# write addresses of shared memory regions to PL registers
reg_space.write(capture_addr_offset, audio_buffer.device_address)

In [6]:
reg_space.write(capture_addr_offset, audio_buffer.device_address)

In [7]:
hex(audio_buffer.device_address)

'0x16900000'

In [8]:
# check debug counters
reg_space.read(lrck_cnt_offset)

3247287

In [9]:
# check debug counters
reg_space.read(bck_cnt_offset)

52926017

In [38]:
reg_space.write(srst_i2s_offset, 1)

In [10]:
# do a capture
reg_space.write(capture_size_offset, capture_size)
reg_space.write(capture_start_offset, 1)
while True:
    done = reg_space.read(transfer_done_offset)
    if done == 1:
        break
    time.sleep(0.1)
audio_buffer.invalidate()
print("successfully captured audio data!")

successfully captured audio data!


In [11]:
# create output wav file
outdir = "/home/xilinx/i2s_to_mp3/audio/"
fname_wav = outdir+"capture.wav"
fname_mp3 = outdir+"capture.mp3"
fs = 44100

samp_left = np.zeros(capture_size, dtype=np.int16)
samp_right = np.zeros(capture_size, dtype=np.int16)
for i in range(capture_size):
    samp_right[i] = np.int16(audio_buffer[i] & 0xFFFF)
    samp_left[i] = np.int16((audio_buffer[i] >> 16) & 0xFFFF)

signal = np.zeros((2,samp_left.shape[0]),dtype=np.int16)
signal[0] = samp_left
signal[1] = samp_right
signal = np.transpose(signal)
    
wavfile.write(fname_wav, fs, signal)
os.system("ffmpeg -i "+fname_wav+" -vn -ar 44100 -ac 2 -b:a 192k "+fname_mp3)

0